In [ ]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [5]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-03-05 19:32:45,911 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 19:32:45,913 INFO: Initializing external client
2025-03-05 19:32:45,914 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 19:32:46,440 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214711
Fetching data from 2025-02-05 00:32:45.911742+00:00 to 2025-03-05 23:32:45.911742+00:00
2025-03-05 19:33:42,851 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "/home/ashish159565/miniconda3/envs/appliedml/lib/python3.11/site-packages/hsfs/core/arrow_flight_client.py", line 209, in __init__
    self._health_check()
  File "/home/ashish159565/miniconda3/envs/appliedml/lib/python3.11/site-packages/retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ashis

AttributeError: 'ArrowFlightClient' object has no attribute '_server_version'

In [ ]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 17:48:28,367 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 17:48:28,373 INFO: Initializing external client
2025-03-05 17:48:28,374 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 17:48:28,845 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214711


In [ ]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [ ]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,216,2.0,2025-03-05 23:00:00+00:00
1,217,0.0,2025-03-05 23:00:00+00:00
2,261,16.0,2025-03-05 23:00:00+00:00
3,65,1.0,2025-03-05 23:00:00+00:00
4,56,0.0,2025-03-05 23:00:00+00:00
...,...,...,...
247,30,0.0,2025-03-05 23:00:00+00:00
248,73,0.0,2025-03-05 23:00:00+00:00
249,35,1.0,2025-03-05 23:00:00+00:00
250,4,4.0,2025-03-05 23:00:00+00:00


In [ ]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-05 17:48:30,497 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 17:48:30,499 INFO: Initializing external client
2025-03-05 17:48:30,500 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-05 17:48:30,979 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214711


In [12]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 252/252 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214711/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)